In [18]:
import numpy as np 
import pandas as pd 
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from numpy import array,argmax
from sklearn.preprocessing import LabelEncoder
# import seaborn as sn

In [19]:
ENetb7_testing_act = np.load('numpy_data/efficientNetb7_testing_actual.npy')
ENetb7_testing_pre = np.load('numpy_data/efficientNetb7_testing_predicted.npy')
ENetb1_testing_pre = np.load('numpy_data/efficientNetb1_testing_predicted.npy')
ENetb1_testing_act = np.load('numpy_data/efficientNetb1_testing_actual.npy')

In [20]:
integer_encoded = ENetb7_testing_act
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
output_vec = onehot_encoder.fit_transform(integer_encoded)

In [21]:
ENetb1_testing_pre_u = []
for i in range(ENetb1_testing_pre.shape[0]):
    ENetb1_testing_pre_u.append(ENetb1_testing_pre[i][0])
    
ENetb7_testing_pre_u = []
for i in range(ENetb7_testing_pre.shape[0]):
    ENetb7_testing_pre_u.append(ENetb7_testing_pre[i][0])

In [22]:
input_vec = np.dstack([ENetb1_testing_pre_u,ENetb7_testing_pre_u])

In [23]:
input_vec[0]

array([[1.0000000e+00, 9.9999976e-01],
       [8.6461185e-11, 1.7408821e-07],
       [1.0897676e-11, 3.0080388e-10],
       [1.2556078e-14, 1.2071452e-09],
       [1.1384914e-14, 9.2887378e-09],
       [4.4942853e-17, 2.0896671e-08],
       [2.9252041e-15, 9.7620740e-12],
       [4.4231835e-20, 9.1602602e-14],
       [5.2400982e-18, 5.2043140e-12],
       [4.8601803e-19, 1.8993693e-10],
       [7.6285488e-17, 2.0801964e-10]], dtype=float32)

In [24]:
input_vec.shape

(3288, 11, 2)

In [28]:
model = tf.keras.Sequential([tf.keras.layers.InputLayer(input_shape=(11,2))
                            ,tf.keras.layers.Dense(units=11, activation=tf.nn.sigmoid),
                             tf.keras.layers.Dense(units=33, activation=tf.nn.sigmoid),
                             tf.keras.layers.Dense(units=11, activation=tf.nn.sigmoid),
                            tf.keras.layers.Dense(units=1, activation=tf.nn.sigmoid),
                            tf.keras.layers.Flatten()])
  

In [29]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 11, 11)            33        
_________________________________________________________________
dense_13 (Dense)             (None, 11, 33)            396       
_________________________________________________________________
dense_14 (Dense)             (None, 11, 11)            374       
_________________________________________________________________
dense_15 (Dense)             (None, 11, 1)             12        
_________________________________________________________________
flatten_3 (Flatten)          (None, 11)                0         
Total params: 815
Trainable params: 815
Non-trainable params: 0
_________________________________________________________________


In [30]:
np.random.seed = 5

In [31]:
model.compile(optimizer='adam', loss='CategoricalCrossentropy',)
model.fit(x=input_vec,y=output_vec,epochs=5)

Epoch 1/5
103/103 [==============================] - 0s 1ms/step - loss: 2.2837
Epoch 2/5
103/103 [==============================] - 0s 1ms/step - loss: 1.1777
Epoch 3/5
103/103 [==============================] - 0s 1ms/step - loss: 0.2149
Epoch 4/5
103/103 [==============================] - 0s 1ms/step - loss: 0.0918
Epoch 5/5
103/103 [==============================] - 0s 1ms/step - loss: 0.0603


## EfficientNetB1 confusion matrix

In [32]:
predicted_class = []
for i in ENetb1_testing_pre:
    predicted_class.append(i.argmax())
actual_class = ENetb1_testing_act
out_df = pd.DataFrame(np.vstack([predicted_class,actual_class]).T,columns=['predicted_class','actual_class'])
confusion_matrix = pd.crosstab(out_df['actual_class'],out_df['predicted_class'], rownames=['Actual'], colnames=['Predicted'])
print(confusion_matrix)
print('Test accuracy : {}'.format((np.diagonal(confusion_matrix).sum()/confusion_matrix.sum().sum()*100)))
out_df.to_csv('B1_model_out.csv')

Predicted   0    1    2    3    4    5    6    7    8    9    10
Actual                                                          
0          345    0    0    0    0    0    0    0    0    0    0
1            0  270    0    0    0    0    0    0    0    0    0
2            0    0  345    0    0    0    0    0    0    0    0
3            0    0    0  331    0    0    0    0    0    0    0
4            0    0    0    0  342    0    3    0    0    0    0
5            0    0    0    2    0  329    0    0    0    0    0
6            0    0    0    0    1    0  256    0    0    0    0
7            0    0    0    0    0    0    0  215    0    0    0
8            0    0    0    0    0    0    0    0  332    3    0
9            0    0    0    0    0    0    0    0    1  214    0
10           0    0    0    0    0    0    0    0    2    7  290
Test accuracy : 99.42214111922141


## EfficientNetB7 confusion matrix


In [33]:
predicted_class = []
for i in ENetb7_testing_pre:
    predicted_class.append(i.argmax())
actual_class = ENetb7_testing_act
out_df = pd.DataFrame(np.vstack([predicted_class,actual_class]).T,columns=['predicted_class','actual_class'])
confusion_matrix = pd.crosstab(out_df['actual_class'],out_df['predicted_class'], rownames=['Actual'], colnames=['Predicted'])
print(confusion_matrix)
print('Test accuracy : {}'.format((np.diagonal(confusion_matrix).sum()/confusion_matrix.sum().sum()*100)))
out_df.to_csv('B7_model_out.csv')

Predicted   0    1    2    3    4    5    6    7    8    9    10
Actual                                                          
0          345    0    0    0    0    0    0    0    0    0    0
1            0  270    0    0    0    0    0    0    0    0    0
2            0    0  345    0    0    0    0    0    0    0    0
3            0    0    0  328    0    1    0    0    0    2    0
4            0    0    0    0  344    0    1    0    0    0    0
5            0    0    0    0    0  331    0    0    0    0    0
6            0    0    0    0    0    0  257    0    0    0    0
7            0    0    0    0    0    0    0  215    0    0    0
8            0    0    0    0    0    0    0    0  335    0    0
9            0    0    0    0    0    0    0    0    5  209    1
10           0    0    0    0    0    0    0    0    6    2  291
Test accuracy : 99.45255474452554


## Combined model confusion matrix


In [34]:
predicted_class = []
for i in range(ENetb7_testing_pre.shape[0]):
    predicted_class.append(model.predict(input_vec[i]).argmax())
actual_class = ENetb7_testing_act
out_df = pd.DataFrame(np.vstack([predicted_class,actual_class]).T,columns=['predicted_class','actual_class'])
confusion_matrix = pd.crosstab(out_df['actual_class'],out_df['predicted_class'], rownames=['Actual'], colnames=['Predicted'])
print(confusion_matrix)
print('Test accuracy : {}'.format((np.diagonal(confusion_matrix).sum()/confusion_matrix.sum().sum()*100)))

Predicted   0    1    2    3    4    5    6    7    8    9    10
Actual                                                          
0          345    0    0    0    0    0    0    0    0    0    0
1            0  270    0    0    0    0    0    0    0    0    0
2            0    0  345    0    0    0    0    0    0    0    0
3            0    0    0  328    0    1    0    0    0    2    0
4            0    0    0    0  344    0    1    0    0    0    0
5            0    0    0    0    0  331    0    0    0    0    0
6            0    0    0    0    0    0  257    0    0    0    0
7            0    0    0    0    0    0    0  215    0    0    0
8            0    0    0    0    0    0    0    0  334    1    0
9            0    0    0    0    0    0    0    0    2  213    0
10           0    0    0    0    0    0    0    0    4    2  293
Test accuracy : 99.60462287104623
